In [177]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchaudio import load
import os

In [178]:
X_train = pd.read_csv('../Files/Standardization/X_Train.csv')
y_train = pd.read_csv('../Files/y_train.csv')
X_test = pd.read_csv('../Files/Standardization/X_Test.csv')
y_test = pd.read_csv('../Files/y_test.csv')

In [179]:
y_train.shape

(856, 2)

In [180]:
# Extract the gender labels from the second column
gender_train = y_train.Gender
gender_test = y_test.Gender

# Check the shape
print(f'Original gender labels shape: {gender_train.shape}, {gender_test.shape}')

# Ensure labels are binary (0 or 1)
# If they are not already binary, use LabelEncoder to convert
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
gender_train = label_encoder.fit_transform(gender_train)
gender_test = label_encoder.transform(gender_test)

# Check the final shapes
print(f'Final gender labels shape: {gender_train.shape}, {gender_test.shape}')


Original gender labels shape: (856,), (129,)
Final gender labels shape: (856,), (129,)


In [181]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the ANN model
ann_model = Sequential([
    Dense(128, input_shape=(39,), activation='relu'),  
    Dropout(0.5),
    Dense(48, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  
])


ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


ann_history = ann_model.fit(X_train, gender_train, epochs=100, batch_size=32, validation_data=(X_test, gender_test))


Epoch 1/100


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5213 - loss: 0.7692 - val_accuracy: 0.5659 - val_loss: 0.6960
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5649 - loss: 0.6900 - val_accuracy: 0.5581 - val_loss: 0.6915
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5543 - loss: 0.6849 - val_accuracy: 0.5426 - val_loss: 0.6916
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5224 - loss: 0.7005 - val_accuracy: 0.6279 - val_loss: 0.6864
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5416 - loss: 0.6930 - val_accuracy: 0.6434 - val_loss: 0.6769
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5806 - loss: 0.6624 - val_accuracy: 0.6744 - val_loss: 0.6691
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5821 - loss: 0.6675 - val_accuracy: 0.6899 - val_loss: 0.6623
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6383 - loss: 0.6408 - val_accuracy: 0.7054 - val_loss: 0.6

In [182]:
import numpy as np

# Convert DataFrame to numpy array
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

# Reshape for CNN
X_train_cnn = X_train_np.reshape(X_train_np.shape[0], 39, 1, 1) 
X_test_cnn = X_test_np.reshape(X_test_np.shape[0], 39, 1, 1)

print(f'X_train_cnn shape: {X_train_cnn.shape}')  # Should print (856, 39, 1, 1)
print(f'X_test_cnn shape: {X_test_cnn.shape}')    # Should print (129, 39, 1, 1)


X_train_cnn shape: (856, 39, 1, 1)
X_test_cnn shape: (129, 39, 1, 1)


In [183]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


cnn_model = Sequential([
    Conv2D(32, (3, 1), activation='relu', input_shape=(39, 1, 1)),  # (height, width, channels)
    MaxPooling2D((2, 1)),
    Conv2D(64, (3, 1), activation='relu'),
    MaxPooling2D((2, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Single output neuron for binary classification
])


cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


cnn_history = cnn_model.fit(X_train_cnn, gender_train, epochs=100, batch_size=32, validation_data=(X_test_cnn, gender_test))


Epoch 1/100


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5302 - loss: 0.6976 - val_accuracy: 0.6667 - val_loss: 0.6726
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5987 - loss: 0.6639 - val_accuracy: 0.6822 - val_loss: 0.6217
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6749 - loss: 0.6291 - val_accuracy: 0.6977 - val_loss: 0.6152
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6903 - loss: 0.6096 - val_accuracy: 0.7054 - val_loss: 0.5618
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7098 - loss: 0.5645 - val_accuracy: 0.7287 - val_loss: 0.5783
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7448 - loss: 0.5604 - val_accuracy: 0.7054 - val_loss: 0.5495
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7600 - loss: 0.5038 - val_accuracy: 0.7519 - val_loss: 0.5195
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7826 - loss: 0.4923 - val_accuracy: 0.7442 - val_loss: 0.5

In [184]:
ann_loss, ann_accuracy = ann_model.evaluate(X_test, gender_test)
print(f'ANN Model Accuracy: {ann_accuracy * 100:.2f}%')


cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, gender_test)
print(f'CNN Model Accuracy: {cnn_accuracy * 100:.2f}%')


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.9012 - loss: 0.3544
ANN Model Accuracy: 92.25%
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.7907 - loss: 1.5606
CNN Model Accuracy: 85.27%
